<a href="https://colab.research.google.com/github/argonne-lcf/ai-science-training-series/blob/main/05_llm_part2/RAGTutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Retrieval Augmented Generation (RAG)

## Overview
*   Motivation for RAG
*   Idea behind RAG
*   Advantages and Disadvantages
*   Implementation to augment question + answer
*   Advanced applications


#### Imagine you went to live under a rock on August 2006. When you come out in 2024, you are asked how many planets revolve around the sun. What would you say?...
![pluto](https://github.com/architvasan/LLMWorkshop/blob/main/rag_images/pluto_planets.jpeg?raw=1)

This is similar to LLMs which are trained with data until a certain point and then asked questions on data they are not trained on. Understandably, LLMs will either be unable to answer or simply hallucinate a probably wrong answer.

###What can be done?

Have the LLM go to the library using **Retrieval Augmented Generation (RAG)**!

RAG involves adding your own data (via a retrieval tool) to the prompt that you pass into a large language model.


![rag architecture](https://github.com/architvasan/LLMWorkshop/blob/main/rag_images/rag-overview.original.png?raw=1)
Image credit: https://scriv.ai/guides/retrieval-augmented-generation-overview/

RAG has been shown to improve LLM prediction accuracy without needing to increase parameter size.

![rag architecture](https://github.com/architvasan/LLMWorkshop/blob/main/rag_images/rag_acc_v_size.png?raw=1)

*Image credit: Yu, Wenhao. "Retrieval-augmented generation across heterogeneous knowledge." Proceedings of the 2022 Conference of the North American Chapter of the Association for Computational Linguistics: Human Language Technologies: Student Research Workshop. 2022.*

RAG also increases explainability by giving the source for information.

![rag architecture](https://github.com/architvasan/LLMWorkshop/blob/main/rag_images/rag_source_locator.png?raw=1)

Image credit: https://ai.stanford.edu/blog/retrieval-based-NLP/

## Advantages and Disadvantages

### Advantages

*   Provides domain specific context
*   Improves predictive performance and reduces hallucinations
*   Does not increase model parameters
*   Less labor intensive than fine-tuning LLMs

### Disadvantages

*   May introduce latency since we are adding a relatively costly search step
*   If your dataset includes private information, you may inadvertently expose another user with this information.
*   The data you want to use needs to be curated and you should decide how the data should be accessed. This adds time for the initial set-up.


#Implementation

### 1. Install + load relevant modules:
*   langchain
*   torch
*   transformers
*   sentence-transformers
*   datasets
*   faiss-cpu  
*   pypdf
*  unstructure[pdf]
*  huggingface_hub (add hf_token)




In [1]:
!pip install langchain==0.1.5
!pip install --quiet langchain_experimental
!pip install torch
!pip install transformers
!pip install faiss-cpu
!pip install pypdf
!pip install sentence-transformers
!pip install unstructured==0.12.3
!pip install unstructured[pdf]==0.12.3
!pip install tiktoken
!pip install huggingface_hub
from huggingface_hub import login

hf_token = "hf_yourtoken"
login(token=hf_token, add_to_git_credential=True)

  Using cached langchain-0.1.5-py3-none-any.whl.metadata (13 kB)
  Using cached langchain_community-0.0.38-py3-none-any.whl.metadata (8.7 kB)
  Using cached langchain_core-0.1.52-py3-none-any.whl.metadata (5.9 kB)
  Using cached langsmith-0.0.92-py3-none-any.whl.metadata (9.9 kB)
INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
  Using cached langchain_community-0.0.37-py3-none-any.whl.metadata (8.7 kB)
  Using cached langchain_community-0.0.36-py3-none-any.whl.metadata (8.7 kB)
  Using cached langchain_community-0.0.35-py3-none-any.whl.metadata (8.7 kB)
  Using cached langchain_community-0.0.34-py3-none-any.whl.metadata (8.5 kB)
  Using cached langchain_community-0.0.33-py3-none-any.whl.metadata (8.5 kB)
  Using cached langchain_community-0.0.32-py3-none-any.whl.metadata (8.5 kB)
  Using cached langchain_community-0.0.31-py3-none-any.whl.metadata (8.4 kB)
INFO: pip is still looki

In [2]:
!pip install -U langchain-community

### 2. Choose a dataset to use and then load it into your code
Here we are using the pdfs loaded in pdfs/. We load this using langchain DirectoryLoader.

We can load multiple types of datasets into this example though the most commonly used are PDFs and websites.

To load websites, we could also use `langchain WebBaseLoader`

In this example, we will consider PDFs and load them in using `langchain DirectoryLoader`.

We host all PDFs at the PDFs directory `ai-science-training-series/05_llm_part2/PDFs`



In [4]:
! git clone https://github.com/argonne-lcf/ai-science-training-series.git

Cloning into 'ai-science-training-series'...
remote: Enumerating objects: 3684, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 3684 (delta 7), reused 21 (delta 2), pack-reused 3653 (from 1)
Receiving objects: 100% (3684/3684), 391.62 MiB | 25.10 MiB/s, done.
Resolving deltas: 100% (1878/1878), done.
Updating files: 100% (417/417), done.


The below code is the langchain based document loader; however, recently this showed failure. We are working to fix it but we have an alternative code to run using bash commands

In [5]:
from langchain.document_loaders import DirectoryLoader
loader = DirectoryLoader('ai-science-training-series/05_llm_part2/PDFs', glob="**/*.pdf", show_progress=True)
documents = loader.load()

  0%|          | 0/7 [00:00<?, ?it/s][nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
100%|██████████| 7/7 [00:36<00:00,  5.27s/it]


In [ ]:
# from pypdf import PdfReader
# x = !ls llm-workshop/tutorials/04-rag/PDFs/*.pdf
# print(x)
# # creating a pdf reader object
# documents = []
# for f in x:
#   reader = PdfReader('llm-workshop/tutorials/04-rag/PDFs/1703.07076.pdf')

#   # printing number of pages in pdf file
#   print(len(reader.pages))

#   # creating a page object
#   page = reader.pages[0]

#   # extracting text from page
#   # print(page.extract_text())
#   documents.append(page.extract_text())

### 3. Now, we need to split our documents into chunks.
We want the embedding to be greater than 1 word but much less than an entire page. This is essential for the similarity search between the query and the document. Essentially, the query will be searched for greatest similarity to embedded chunks in the dataset. Then those chunks with greatest similarity are augmented to the query.

It is essential to choose the chunking method according to your data type.
There are different ways to do this:

Fixed size
*   Token: Splits text on tokens. Can chunk tokens together
*   Character: Splits based on some user defined character.

Recursive
*  Recursively splits text. Useful for keeping related pieces of text next to each other.

Document based
*   HTML: Splits text based on HTML-specific characters.
*   Markdown: Splits on Markdown-specific characters
*   Code: Splits text based on characters specific to coding languages.

Semantic chunking
*   Extract semantic meaning from embeddings and then assess the semantic relationship between these chunks. Essentially splits into sentences, then groups into groups of 3 sentences, and then merges one that are similar in the embedding space.

Here we use recursive where the dataset is split using a set of characters. The default characters provided to it are ["\n\n", "\n", " ", ""].  A large text is split by the first character \n\n. If the first split by \n\n is still large then it moves to the next character which is \n and tries to split by it. This continues until the chunk size is reached.


In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
#documents = text_splitter.create_documents(documents)
docs = text_splitter.split_documents(documents)

In [7]:
print(docs)

[Document(metadata={'source': 'ai-science-training-series/05_llm_part2/PDFs/s41586-023-06415-8.pdf'}, page_content='De novo design of protein structure and function with RFdiffusion\n\nhttps://doi.org/10.1038/s41586-023-06415-8\n\nReceived: 14 December 2022\n\nAccepted: 7 July 2023\n\nPublished online: 11 July 2023\n\nOpen access\n\nJoseph L. Watson1,2,15, David Juergens1,2,3,15, Nathaniel R. Bennett1,2,3,15, Brian L. Trippe2,4,5,15, Jason Yim2,6,15, Helen E. Eisenach1,2,15, Woody Ahern1,2,7,15, Andrew J. Borst1,2, Robert J. Ragotte1,2, Lukas F. Milles1,2, Basile I. M. Wicky1,2, Nikita Hanikel1,2, Samuel J. Pellock1,2, Alexis Courbet1,2,8, William Sheffler1,2, Jue Wang1,2, Preetham Venkatesh1,2,9, Isaac Sappington1,2,9, Susana Vázquez Torres1,2,9, Anna Lauko1,2,9, Valentin De Bortoli8, Emile Mathieu10, Sergey Ovchinnikov11,12, Regina Barzilay6, Tommi S. Jaakkola6, Frank DiMaio1,2, Minkyung Baek13 & David Baker1,2,14\u2009✉\n\nCheck for updates'), Document(metadata={'source': 'ai-scienc

In [ ]:
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
# docs = text_splitter.split_documents(documents)

AttributeError: 'str' object has no attribute 'page_content'

### 4. Then we embed the chunked texts using a Transformer.
This allows us to encode the text into our search

Embedding converts text to a numerical representation in a vector space. RAG compares the embeddings of user queries within the vector of the knowledge library.

In this example, we choose a simple embedding using the MiniLM

In [8]:
from langchain.embeddings import HuggingFaceEmbeddings
modelPath = "sentence-transformers/all-MiniLM-l6-v2"
model_kwargs = {'device':'cpu'}
encode_kwargs = {'normalize_embeddings':False}
embeddings = HuggingFaceEmbeddings(
  model_name = modelPath,
  model_kwargs = model_kwargs,
  encode_kwargs=encode_kwargs
)

<ipython-input-8-adf0a18b0ba1>:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

### 5.Create a vector database
Vector databases, also called vector storage, efficiently store and retrieve vector data, which are arrays of numerical values representing points in multi-dimensional space. They're useful for handling data like embeddings from deep learning models or numerical features. Unlike traditional relational databases, which aren't optimized for vectors, vector databases offer efficient storage, indexing, and querying for high-dimensional and variable-length vectors.

There are various types of vector databases:
1. Chroma
2. FAISS
3. Pinecone
4. Weaviate
5. Qdrant

Here, we build this using the FAISS utility.

![vector_database](https://github.com/argonne-lcf/llm-workshop/blob/main/tutorials/04-rag/rag_images/vector_database.png?raw=1)

Image credit: https://blog.gopenai.com/primer-on-vector-databases-and-retrieval-augmented-generation-rag-using-langchain-pinecone-37a27fb10546

In [9]:
from langchain.vectorstores import FAISS
db = FAISS.from_documents(docs, embeddings)
question = "What is RF Fold?"
searchDocs = db.similarity_search(question)
print(searchDocs[0].page_content)

RFdiffusion, allowing it to efficiently target this site (right bar, pink). C-D) As well as conditioning on hotspot residue information, a fine-tuned RFdiffusion model can also condition on input fold information (secondary structure and block-adjacency information - see Supplementary Methods 4.5). This effectively allows the specification of a (for instance, particularly compatible) fold that the binder should adopt. C) Two examples showing binders can be specified to adopt either a ferredoxin fold (left) or a particular helical bundle fold (right). D) Quantification of the efficiency of fold-conditioning. Secondary structure inputs were accurately respected (top, pink). Note that in this design target and target site, RFdiffusion without fold-specification made generally helical


### 6. Initialize the LLM that will be used for question answering

Here, we use a pretrained model flan-t5-large as part of a HuggingFacePipeline. This will later be chained with the vector database for RAG.

In [10]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM,pipeline
from langchain import HuggingFacePipeline

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large")
pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer)
llm = HuggingFacePipeline(
   pipeline = pipe,
   model_kwargs={"temperature": 0, "max_length": 2048},
)
#'HuggingFaceH4/zephyr-7b-beta'

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
<ipython-input-10-a5162468da6c>:7: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(


### 7. Retrieve data and use it to answer a question

![rag_workflow](https://github.com/argonne-lcf/llm-workshop/blob/main/tutorials/04-rag/rag_images/rag_workflow.png?raw=1)

Image credit: https://blog.gopenai.com/retrieval-augmented-generation-101-de05e5dc21ef

Let's ask questions it would only be able to know if the model actually read the texts!

In [11]:
from langchain.prompts import PromptTemplate

template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Keep the answer as concise as possible.
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

In [12]:
from langchain.chains import RetrievalQA
qa_chain = RetrievalQA.from_chain_type(
  llm=llm,
  chain_type="stuff",
  retriever=db.as_retriever(),
  chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)
result = qa_chain ({ "query" : "What technique proposed in 2023 can be used to predict protein folding?" })
print(result["result"])

<ipython-input-12-8e78b448a1f8>:8: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain ({ "query" : "What technique proposed in 2023 can be used to predict protein folding?" })
Token indices sequence length is longer than the specified maximum sequence length for this model (1076 > 512). Running this sequence through the model will result in indexing errors
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


RFdiffusion


Now let's ask the chain where to find the article related to RFDiffusion

In [13]:
qa_chain ({ "query" : "Which scientific article should I read to learn about RFdiffusion for protein folding?" })

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


{'query': 'Which scientific article should I read to learn about RFdiffusion for protein folding?',
 'result': 'Nature | Vol 620 | 31 August 2023 | 1091'}

## Exercise

Use any of the frameworks/models here to load in your favorite websites and ask the model a question regarding them.

Hint:
```
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader(["https://www.espn.com/", "https://www.google.com"])

To bypass SSL verification errors during fetching, you can set the “verify” option:

loader.requests_kwargs = {‘verify’:False}

data = loader.load()
```

## More applications

### RAG using Llama 2, Langchain and ChromaDB

In [ ]:
!pip install einops xformers \
bitsandbytes chromadb

Defaulting to user installation because normal site-packages is not writeable
  Using cached einops-0.8.0-py3-none-any.whl.metadata (12 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached requests_oauthlib-2.0.0-py2.py3-none-any.whl.metadata (11 kB)
  Using cached monotonic-1.6-py2.py3-none-any.whl.metadata (1.5 kB)
Using cached einops-0.8.0-py3-none-any.whl (43 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 93.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 6.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 15.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 114.2 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.8 MB/s eta 0:00:00:00:0100:0

In [ ]:
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.7/279.7 kB 7.6 MB/s eta 0:00:00


Load in models and setup pipeline

In [ ]:
from torch import cuda, bfloat16
import torch
import transformers
from transformers import AutoTokenizer
from time import time
#import chromadb
#from chromadb.config import Settings
from langchain.llms import HuggingFacePipeline
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma

model_id = 'meta-llama/Llama-2-7b-chat-hf'

device = f'cuda'

# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=False,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

In [ ]:
time_1 = time()
model_config = transformers.AutoConfig.from_pretrained(
    model_id,
)
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
)
tokenizer = AutoTokenizer.from_pretrained(model_id)
time_2 = time()
print(f"Prepare model, tokenizer: {round(time_2-time_1, 3)} sec.")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

Prepare model, tokenizer: 73.339 sec.


In [ ]:
time_1 = time()
query_pipeline = transformers.pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        torch_dtype=torch.float16,
        device_map="auto",)
time_2 = time()
print(f"Prepare pipeline: {round(time_2-time_1, 3)} sec.")

Prepare pipeline: 12.021 sec.


Prepare function to test pipeline

In [ ]:
def test_model(tokenizer, pipeline, prompt_to_test):
    """
    Perform a query
    print the result
    Args:
        tokenizer: the tokenizer
        pipeline: the pipeline
        prompt_to_test: the prompt
    Returns
        None
    """
    # adapted from https://huggingface.co/blog/llama2#using-transformers
    time_1 = time()
    sequences = pipeline(
        prompt_to_test,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=200,)
    time_2 = time()
    print(f"Test inference: {round(time_2-time_1, 3)} sec.")
    for seq in sequences:
        print(f"Result: {seq['generated_text']}")

In [ ]:
test_model(tokenizer,
           query_pipeline,
           "Please explain what is the State of the Union address. Give just a definition. Keep it in 100 words.")


Test inference: 732.623 sec.
Result: Please explain what is the State of the Union address. Give just a definition. Keep it in 100 words.
The State of the Union address is an annual speech delivered by the President of the United States to Congress, in which the President reports on the state of the union and outlines legislative priorities for the upcoming year.


Set up Huggingface pipeline

In [ ]:
llm = HuggingFacePipeline(pipeline=query_pipeline)
# checking again that everything is working fine
llm(prompt="Please explain what is the State of the Union address. Give just a definition. Keep it in 100 words.")

NameError: name 'HuggingFacePipeline' is not defined

Load in data

In [ ]:
loader = TextLoader("llm-workshop/tutorials/04-rag/state_union2023.txt",
                    encoding="utf8")
documents = loader.load()

Chunk data recursively

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
all_splits = text_splitter.split_documents(documents)

In [ ]:
all_splits

[Document(page_content='Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.\n\nLast year COVID-19 kept us apart. This year we are finally together again.\n\nTonight, we meet as Democrats Republicans and Independents. But most importantly as Americans.\n\nWith a duty to one another to the American people to the Constitution.\n\nAnd with an unwavering resolve that freedom will always triumph over tyranny.\n\nSix days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated.\n\nHe thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined.\n\nHe met the Ukrainian people.\n\nBiden condemns Putin in scathing State of the Union speech, in 180 seconds', metadata={'source': 'llm-workshop/tutorials/04-rag/state_union2023.txt'})

Embed and store in Chroma Vector store

In [ ]:
model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device':'cpu'}

embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)

In [ ]:
vectordb = Chroma.from_documents(documents=all_splits, embedding=embeddings, persist_directory="chroma_db")

Set up chain

In [ ]:
retriever = vectordb.as_retriever()

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    verbose=True
)

Test RAG

In [ ]:
def test_rag(qa, query):
    print(f"Query: {query}\n")
    time_1 = time()
    result = qa.run(query)
    time_2 = time()
    print(f"Inference time: {round(time_2-time_1, 3)} sec.")
    print("\nResult: ", result)

Get sources...

In [ ]:
docs = vectordb.similarity_search(query)
print(f"Query: {query}")
print(f"Retrieved documents: {len(docs)}")
for doc in docs:
    doc_details = doc.to_json()['kwargs']
    print("Source: ", doc_details['metadata']['source'])
    print("Text: ", doc_details['page_content'], "\n")

In [ ]:
query = "What were the main topics in the State of the Union in 2023? Summarize. Keep it under 200 words."
test_rag(qa, query)